## Advanced usage
### Using config files

Instead of specifying all inputs using [set_input](https://inbo.github.io/niche_vlaanderen/lowlevel.html#niche_vlaanderen.Niche.set_input), it is possible to use a config file. A config file can be loaded using [read_config_file](https://inbo.github.io/niche_vlaanderen/lowlevel.html#niche_vlaanderen.Niche.read_config_file) or it can be read and executed immediately by using [run_config_file](https://inbo.github.io/niche_vlaanderen/lowlevel.html#niche_vlaanderen.Niche.run_config_file).

The syntax of the config file is explained more in detail in [Niche Configuration file](https://inbo.github.io/niche_vlaanderen/cli.html), but is already introduced here because it will be used in the next examples.


### Comparing Niche classes

Niche models can be compared using a [NicheDelta](lowlevel.rst#niche_vlaanderen.NicheDelta) class. This can be used to compare different scenario's. 

In our example, we will compare the results of the running Niche two times, once using a simple model and once using a full model. 

In [ ]:
import niche_vlaanderen as nv
import matplotlib.pyplot as plt

simple = nv.Niche()
simple.run_config_file("simple.yml")

full = nv.Niche()
full.run_config_file("full.yml")

delta = nv.NicheDelta(simple, full)
ax = delta.plot(7)
plt.show()

It is also possible to show the areas in a dataframe by using the [table](lowlevel.rst#niche_vlaanderen.NicheDelta.table) attribute.

In [ ]:
delta.table.head()

Like Niche, NicheDelta also has a write method, which takes a directory as an argument.

In [ ]:
delta.write("comparison_output", overwrite_files=True)

### Creating deviation maps

In many cases, it is not only important to find out which vegetation types are possible given the different input files, but also to find out how much change would be required to `mhw` or `mlw` to allow a certain vegetation type.

To create deviation maps, it is necessary to [run](lowlevel.rst#niche_vlaanderen.Niche.run) a model with the `deviation` option.

In [ ]:
dev = nv.Niche()
dev.set_input("mhw","../testcase/zwarte_beek/input/mhw.asc")
dev.set_input("mlw","../testcase/zwarte_beek/input/mhw.asc")
dev.set_input("soil_code","../testcase/zwarte_beek/input/soil_code.asc")
dev.run(deviation=True, full_model=False)

The deviation maps can be plotted by specifying either mhw or mlw with the vegetation type, eg mhw_14 (to show the deviation between mhw and the required mhw for vegetation type 14).
Positive values indicate that the actual condition is too dry for the vegetation type. Negative values indicate that the actual condition is too wet for the vegetation type. 

In [ ]:
dev.plot("mlw")
dev.plot("mlw_14")

plt.show()

### Creating statistics per shape object

Niche also contains a helper function that allows one to calculate the possible vegetation by using a vector dataset, such as a .geojson or .shp file.

The vegetation is returned as a dict of pandas dataframes (one for every shape).

In [ ]:
df = full.zonal_stats("../testcase/zwarte_beek/input/study_area_l72.geojson")
df

### Using abiotic grids

In certain cases the intermediary grids of Acidity or NutrientLevel need changes, to compensate for specific circumstances.

In that case it is possible to run a Niche model and make some adjustments to the grid and then using an abiotic grid as an input.



In [ ]:
import niche_vlaanderen as nv
import matplotlib.pyplot as plt

full = nv.Niche()
full.run_config_file("full.yml")
full.write("output_abiotic", overwrite_files=True)

Now it is possible to adapt the `acidity` and `nutrient_level` grids outside niche. For this demo, we will use some Python magic to make all nutrient levels one level lower. Note that there is no need to do this in Python, any other tool could be used as well. So if you don't understand this code - don't panic (and ignore the warning)!

In [ ]:
import rasterio
with rasterio.open("output_abiotic/full_nutrient_level.tif") as src:
    nutrient_level = src.read(1)
    profile = src.profile
    nodata = src.nodatavals[0]
    
nutrient_level[nutrient_level != nodata] = nutrient_level[nutrient_level != nodata] -1

# we can not have nutrient level 0, so we set all places where this occurs to 1
nutrient_level[nutrient_level ==0 ] = 1

with rasterio.open("output_abiotic/adjusted_nutrient.tif", 'w', **profile) as dst:
    dst.write(nutrient_level, 1)

Next we will create a new niche model using the same options as our previous full models, but we will also add the previously calculated acidity and nutrient level values as input, and run with the `abiotic=True` option. Note that we use the `read_config_file` method (and not  `run_config_file`) because we still want to edit the configuration before running.

In [ ]:
adjusted = nv.Niche()
adjusted.read_config_file("full.yml")
adjusted.set_input("acidity", "output_abiotic/full_acidity.tif")
adjusted.set_input("nutrient_level", "output_abiotic/adjusted_nutrient.tif")
adjusted.name = "adjusted"
adjusted.run(abiotic=True)

adjusted.plot(7)
full.plot(7)
plt.show()